In [1]:
import pandas
import json
from tqdm import tqdm
import re
import os
import requests
import datetime
from bs4 import BeautifulSoup as bs
from collections import defaultdict

In [2]:
def printer(data):
    """
    get: list of dictionaries
    print: 'title' and 'link' from each dictionari
    """
    for e in data:
        print(e['title'])
        print(e['href'])
        
        
def finder(data, select):
    """
    get: list of dictionaries
    return: list of dictionaries minus sublist of dictionaries by selector,
            sublist of dictionaries by selector
    """
    g = []
    v = []
    
    for e in data:
        t = e['title']
        if (re.match(r'[ДПА][а-яВ\s]+\d{,2}[\.\s][\dа-я]+[\.\s]\d{,4}', t)
            #юшпотВиыдлрквнеьйаугямжс
            and select in t):
            g.append(e)
        else:
            v.append(e)
    return v, g


def xa0(string):
    """
    Hey, mister \xa0, GTFOH! 
    """
    if '\xa0' in string:
        string = string.replace('\xa0', '')
    return string


def check_keys(dictofdicts):
    """
    get: dictionari of dictionaries
    print: keys of dictionaries
    """
    q = []
    for k, v in w.items():
        for i, l in v.items():
            q.append(i)
    print(set(q))
    
    
def get_date(yyyy, mm, dd):
    """
    get: mess
    return: DateTime format
    """
    mm_eng = {'Янв':'Jan', 'Фев':'Feb', 'Мар':'Mar', 'Апр':'Apr', 
              'Мая':'May', 'Июн':'Jun', 'Июл':'Jul', 'Авг':'Aug', 
              'Сен':'Sep', 'Окт':'Oct', 'Ноя':'Nov', 'Дек':'Dec'}
    
    g = dd+'/'+mm_eng[mm.capitalize()]+'/'+yyyy
    d = datetime.datetime.strptime(g, '%d/%b/%Y').date()
    return d


def DieWalkuere():
    """
    return: json from career.hse.ru/news
    """

    m_r = requests.get('https://career.hse.ru/news/')
    soup = bs(m_r.text, 'lxml')
    num_of_cycles = int(soup.find_all('a', {'class' : 'pages__page'})[-1].text)
    hse_career_posts = []

    with tqdm(total=num_of_cycles) as pbar:
        for i in range(num_of_cycles):
            r = requests.get(f'https://career.hse.ru/news/page{i}.html')
            soup = bs(r.text, 'lxml')

            posts_content = soup.find_all("div", {"class": "post__content"})
            posts_meta = soup.find_all("div", {"class": "post-meta__date"})

            qq = list(zip(posts_content, posts_meta))

            for e in qq:
                title = e[0].find('a').text
                href = e[0].find('a')['href']
                text_content = e[0].find('div', {'class' : 'post__text'}).text

                dd = e[1].find('div', {'class' : 'post-meta__day'}).text
                mm = e[1].find('div', {'class' : 'post-meta__month'}).text
                yyyy = e[1].find('div', {'class' : 'post-meta__year'}).text

                d = str(get_date(yyyy, mm, dd))

                e_dict = {'title':title, 'href':href, 
                          'text_content':text_content, 'date':d}
                hse_career_posts.append(e_dict)
            pbar.update() 

    with tqdm(total=len(hse_career_posts)) as pbar:
        for e in hse_career_posts:
            try:
                url = e['href']
                if 'http:' in url:
                    url = url.replace('http:', 'https:')
                if not 'https:' in url:
                    url = 'https:'+url
                r = requests.get(url).text
                e['href_content_html'] = r
            except:
                e['href_content_html'] = ''
                pass
            pbar.update()
    return hse_career_posts


def Fricka(listofobj):
    """
    get: list of dictionaries
    return: list of strings
    """
    title_p = "margin-bottom: .0001pt; text-align: center; text-indent: 1.0cm; line-height: 150%;"
    Liste = []
    with tqdm(total=len(listofobj)) as pbar:
        for obj in listofobj:
            href = obj.get('href_content_html')
            o = bs(href, 'lxml')
            core_div = o.find('div', {'class': 'post__text'})
            list_of_p = core_div.find_all('p')
            for p in list_of_p:
                if p.get('style') == title_p:
                    tex = p.text
                    tex = xa0(tex)
                    if tex != '':
                        Liste.append('o'+tex)
                        Liste.append(obj['date'])
                else:
                    spans = p.find_all('span')
                    for s in spans:
                        tex = s.text
                        tex = xa0(tex)
                        if tex != '':
                            Liste.append(tex)
            pbar.update()
    return Liste


def Wotan(listofstr):
    """
    get: list of strings
    return: dictionari of dictionaries
            example: {title_1 : {aspect_1 : text_1, aspect_2 : text_2}, 
                      title_2 : {aspect_1 : text_1, aspect_2 : text_2},
                      ...
                      }
    """
    
    keys = ['Контакты:', 'О компании:', 'Обязанности:', 'Условия:', 'Окомпании:', 'Требования:']
    Woerterbuch = defaultdict(dict)
    current_date = ''
    current_title = ''
    curretn_aspect = ''
    
    with tqdm(total=len(listofstr)) as pbar:
        for i, e in enumerate(listofstr):
            t = re.compile(r'[o]\d+[\.][\s]\b')
            if t.match(e):
                nomore = t.match(e).group()
                current_title = e.replace(nomore, '')
                Woerterbuch[current_title]
                #print(i, current_title)
            try:   
                if re.match(r'\d{4}[-]\d{2}[-]\d{2}\b', e):
                    current_date = e
                    Woerterbuch[current_title]['date'] = current_date
                    #print(i, current_date)
                elif e.strip() in keys:
                    curretn_aspect = e.strip()
                    if curretn_aspect == 'Окомпании:':
                        curretn_aspect = 'О компании:'
                    Woerterbuch[current_title][curretn_aspect] = ''
                    #print(i, curretn_aspect)
                else:
                    Woerterbuch[current_title][curretn_aspect]+=e
                
            except:
                pass
            pbar.update() 
    return Woerterbuch


def Bruennhilde(dictofdicts):
    """
    get: dictionary of dictionaries
            example: {title_1 : {aspect_1 : text_1, aspect_2 : text_2}, 
                      title_2 : {aspect_1 : text_1, aspect_2 : text_2},
                      ...
                      }
    return: list of dictionaries
            example: [{title : title_1, aspect_1 : text_1, aspect_2 : text_2}, 
                      {title : title_2, aspect_1 : text_1, aspect_2 : text_2},
                      ...
                      ]
    """
    with open('metro.txt', 'r', encoding='utf-8') as f:
        ms = f.read().split('\n')
    
    Siegfried = []
    for k, v in dictofdicts.items():
        Fafnir = {}
        c = k.split(' в ')
        n = re.compile(r'[А-ЯЁA-Z][\w\s]+')
        if len(c) > 1 and n.search(c[-1]):
            Fafnir['title'] = k
            Fafnir['company'] = n.search(c[-1]).group()
        else:
            Fafnir['title'] = k
        money = re.compile(r'[\d][\d\s]+[руб]{1,3}[.]{,1}\s')
        metrore = re.compile(r'\s[м][.][\s][А-ЯЁ][а-я]+\s')
        if 'Условия:' in v:
            cond = v['Условия:']
            if money.search(cond):
                Fafnir['money'] = ', '.join(money.findall(cond))
            metro = []
            if metrore.search(cond):
                for e in ms:
                    if e in cond:
                        metro.append('м. ' + e)
                if len(metro) > 0:
                    Fafnir['metro'] = ', '.join(metro)
            Fafnir.update(v)
        Siegfried.append(Fafnir)
    return Siegfried

In [ ]:
#This cell downloads the website, be careful!

Data = DieWalkuere()

with open('hse_career_department_website_posts.json', 'w') as f:
        json.dump(Data, f)

In [3]:
with open('hse_career_department_website_posts.json', 'r') as f:
        Data = json.load(f)

In [4]:
data2, stud = finder(Data, 'студент')

In [5]:
data3, grad = finder(data2, 'выпускник')

In [6]:
step1_stud = Fricka(stud)
step2_stud = Wotan(step1_stud)
step3_stud = Bruennhilde(step2_stud)

Table_stud = pandas.DataFrame(step3_stud)

100%|████████████████████████████████████████████████████████████████████████| 30505/30505 [00:00<00:00, 147236.39it/s]


In [7]:
step1_grad = Fricka(grad)
step2_grad = Wotan(step1_grad)
step3_grad = Bruennhilde(step2_grad)

Table_grad = pandas.DataFrame(step3_grad)

100%|████████████████████████████████████████████████████████████████████████| 22616/22616 [00:00<00:00, 136976.25it/s]


In [ ]:
#Table_stud

In [ ]:
#Table_grad

In [8]:
os.mkdir('results')

with open('results\Table_stud.json', 'w') as f:
        json.dump(data3, f)
Table_stud.to_csv('results\Table_stud.csv', encoding='utf-8')


with open('results\Table_grad.json', 'w') as f:
        json.dump(data3, f)
Table_grad.to_csv('results\Table_grad.csv', encoding='utf-8')


with open('results\ostatki.json', 'w') as f:
        json.dump(data3, f)